In [1]:
import peewee
from peewee import *
import pandas as pd
from mlxtend.preprocessing import OnehotTransactions
from mlxtend.frequent_patterns import association_rules
from collections import defaultdict
from sklearn import preprocessing
from mlxtend.frequent_patterns import apriori
from tabulate import tabulate



In [2]:
 db=MySQLDatabase('backery',user='root',passwd='password')
class BaseModel(peewee.Model):
    class Meta:
        database = db 
class employee(BaseModel):
    Last=peewee.CharField()
    First=peewee.CharField()
    HireDate=peewee.DateField()
    FireDate=peewee.DateField()
    Position=peewee.DateField()
    FullTime=peewee.DateField()
    StoreNum=peewee.BigIntegerField()
    EmpId=peewee.BigIntegerField()
#     def attr(self):
#         return list(self.Last, self.First,self.Position,self.FullTime,self.StoreNum,self.EmpId)
    def __str__(self):
        return "({0}, {1}, {2} , {3}, {4}, {5})".format(self.Last, self.First,self.Position,
                                                        self.FullTime,self.StoreNum, self.EmpId)
    
class goods(BaseModel):
    Id=peewee.BigIntegerField()
    Flavor=peewee.CharField()
    Food=peewee.CharField()
    Price=peewee.FloatField()
    Type=peewee.CharField()
class item(BaseModel):    
    Receipt=peewee.BigIntegerField()
    Quantity=peewee.BigIntegerField()
    Item=peewee.BigIntegerField()
class location(BaseModel):
    City=peewee.CharField()
    State=peewee.CharField()
    Zip=peewee.BigIntegerField()
    Street=peewee.CharField()
    StoreNum=peewee.BigIntegerField()
class recepits(BaseModel):
    ReceiptNumber=peewee.BigIntegerField()
    SaleDate=peewee.DateField()
    Weekend=peewee.CharField()
    isCash=peewee.CharField()
    EmpId=peewee.BigIntegerField()
    StoreNum=peewee.BigIntegerField()
db.connect()
db.get_tables()
employees=employee.select()
allrows = db.execute_sql('Select * from employee,location,goods,items,receipts where employee.StoreNum=\
location.StoreNum and employee.StoreNum=receipts.StoreNum and receipts.ReceiptNumber=items.Receipt and items.Item=\
goods.Id limit 10')

# save into a data frame having duplicated columns though, thereby rename is required, ie. to employee.StoreNum
# vs location.StoreNum
# use pandas for csv creation or just try ahead with spark
 for row in allrows:
       print(employee)

<class '__main__.employee'>
<class '__main__.employee'>
<class '__main__.employee'>
<class '__main__.employee'>
<class '__main__.employee'>
<class '__main__.employee'>
<class '__main__.employee'>
<class '__main__.employee'>
<class '__main__.employee'>
<class '__main__.employee'>


In [3]:
allrows


In [4]:
%spark

val prop = new java.util.Properties
prop.setProperty("user","root")
prop.setProperty("password","password")
val url="jdbc:mysql://localhost:3306/backery"

val receipts=spark.sqlContext.read.jdbc(url,"receipts",prop)
val items=spark.sqlContext.read.jdbc(url,"items",prop)
val goods=spark.sqlContext.read.jdbc(url,"goods",prop)
val location=spark.sqlContext.read.jdbc(url,"location",prop)
val employee=spark.sqlContext.read.jdbc(url,"employee",prop)

val joinedDF=employee.join(location,"StoreNum").join(receipts.drop("EmpId"),"StoreNum").join(items,receipts("ReceiptNumber")===items("Receipt")).join(goods,items("Item")===goods("Id"))

scala> joinedDF.printSchema()
root
 |-- StoreNum: integer (nullable = true)
 |-- Last: string (nullable = true)
 |-- First: string (nullable = true)
 |-- HireDate: date (nullable = true)
 |-- FireDate: date (nullable = true)
 |-- Position: string (nullable = true)
 |-- FullTime: string (nullable = true)
 |-- EmpId: integer (nullable = false)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zip: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- ReceiptNumber: integer (nullable = false)
 |-- SaleDate: date (nullable = true)
 |-- Weekend: string (nullable = true)
 |-- isCash: string (nullable = true)
 |-- Receipt: integer (nullable = false)
 |-- Quantity: integer (nullable = true)
 |-- Item: integer (nullable = false)
 |-- Id: integer (nullable = false)
 |-- Flavor: string (nullable = true)
 |-- Food: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Type: string (nullable = true)
    
joinedDF.coalesce(1).write.format("csv").option("header","true").save("bakery.csv")


SyntaxError: invalid syntax (<ipython-input-4-9423376c35d2>, line 3)

In [7]:
df=pd.read_csv('./part-00000-cc418b10-1db3-4cde-af1b-9ba3daf80729-c000.csv',sep=',')

print(df.dtypes)
print(df.iloc[1])
df.describe().columns
df.head()

StoreNum           int64
Last              object
First             object
HireDate          object
FireDate          object
Position          object
FullTime            bool
EmpId              int64
City              object
State             object
Zip                int64
Street            object
ReceiptNumber      int64
SaleDate          object
Weekend             bool
isCash              bool
Receipt            int64
Quantity           int64
Item               int64
Id                 int64
Flavor            object
Food              object
Price            float64
Type              object
dtype: object
StoreNum                    14
Last                    CRATCH
First                   RODNEY
HireDate            2000-01-03
FireDate            2000-01-17
Position               Manager
FullTime                  True
EmpId                       14
City                   Seattle
State                       WA
Zip                      98111
Street           550 Thomas St
ReceiptNumber 

,StoreNum,Last,First,HireDate,FireDate,Position,FullTime,EmpId,City,State,...,Weekend,isCash,Receipt,Quantity,Item,Id,Flavor,Food,Price,Type
0,11,STILNER,TOVA,2000-01-19,2000-01-27,Manager,True,11,Sacramento,CA,...,False,True,496,4,31,31,Apple,Croissant,1.45,Food
1,14,CRATCH,RODNEY,2000-01-03,2000-01-17,Manager,True,14,Seattle,WA,...,True,False,243,2,31,31,Apple,Croissant,1.45,Food
2,14,SEINER,CLEMENTE,2000-01-26,2000-01-29,Barista,False,41,Seattle,WA,...,True,False,243,2,31,31,Apple,Croissant,1.45,Food
3,14,ROLLINGER,SANTOS,2000-01-13,2000-01-25,Shift Manager,True,42,Seattle,WA,...,True,False,243,2,31,31,Apple,Croissant,1.45,Food
4,14,JI,STARLA,2000-01-07,2000-01-12,Shift Manager,True,43,Seattle,WA,...,True,False,243,2,31,31,Apple,Croissant,1.45,Food


# Apriori using Python

In [8]:
# changeColNames=dict(enumerate(df.columns))

leCols=LabelEncoder()
leCols.fit_transform(df.columns)
leCity=LabelEncoder()
leCity.fit_transform(df.City.values)
# encodes all non numerical variables, mind that would be useful when justifying rules assoc.
# df[df.columns.difference(df.describe().columns)]=df[df.columns.difference(df.describe().columns)].apply(LabelEncoder().fit_transform)

# print(df.City.values,leCity.inverse_transform(df.City.values))

df.head()

NameError: name 'LabelEncoder' is not defined

# Apriori by Stores, States, City, Food & Flavour

In [9]:
def getRules(dfFilt):
    frequent_itemsets = apriori(dfFilt, use_colnames=True)
    frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
#     return frequent_itemsets[ (frequent_itemsets['length'] > 1) & (frequent_itemsets['support'] >= 0.8)]
#     print(frequent_itemsets[ (frequent_itemsets['length'] > 1) & (frequent_itemsets['support'] >= 0.8)])
    
    rules= association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
    rules["antecedant_len"] = rules["antecedants"].apply(lambda x: len(x))
    return rules[ (rules['antecedant_len'] >= 2) &(rules['confidence'] > 0.75) &(rules['lift'] > 1) ]
    
def rulesByStore(stores):
    global df
    result={}
    for store in stores:
#         print(i)
        dfFilt=df[df['StoreNum']==store]
        result[store]=getRules(dfFilt)
    return result

def rulesByState(states):
    global df
    result={}
    for state in states:
        dfFilt=df[df['State']==state]
        result[state]=getRules(dfFilt)
    return result

def rulesByCity(cities):
    global df
    result={}
    for city in cities:
        dfFilt=df[df['City']==city]
        result[city]=getRules(dfFilt)
    return result
def rulesByFood(foods):
    global df
    result={}
    for food in foods:
        dfFilt=df[df['Food']==food]
        result[food]=getRules(dfFilt)
    return result

# results=rulesByStore([1,2])
results=rulesByStore(df.StoreNum.unique())
for i in results.keys():
    if(len(results[i])>0):
        print(i,tabulate(results[i], headers='keys', tablefmt='psql'))

TypeError: unsupported operand type(s) for /: 'str' and 'float'

# df['Position']=df['Position'].apply(lambda x: x.replace(' ',''))
df['City']=df['City'].apply(lambda x: x.replace(' ',''))
df['Street']=df['Street'].apply(lambda x: x.replace(' ',''))
# df=df.drop('ReceiptNumber',1)
df.columns